Install necessary libraries

In [ ]:
!pip install opencv-python
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 25.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyl

Import python modules

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

Import specific video for landmark extraction

In [ ]:
cap = cv2.VideoCapture("/content/1.mp4")

Visualisation of landmark extraction

In [ ]:
from google.colab.patches import cv2_imshow
import numpy as np
import cv2
import mediapipe as mp

mp_pose = mp.solutions.pose
mp_draw = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Initialise a frame counter
frame_counter = 0

while True:
    # Read frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Error reading frame or end of video. Exiting...")
        break

    # Increment the frame counter
    frame_counter += 1

    # Process only every 5th frame
    if frame_counter % 5 == 0:
        results = pose.process(frame)
        if results.pose_landmarks:
            # Extract only the landmarks of interest
            specific_landmarks = [results.pose_landmarks.landmark[i] for i in [0, 12, 11, 23, 24, 25, 26, 27, 28,15,16,13,14]]

            # Store the landmark labels
            labels = ['Nose', 'Right Shoulder', 'Left Shoulder', 'Left Hip', 'Right Hip', 'Left Knee', 'Right Knee', 'Left Foot', 'Right Foot','Left Wrist','Right Wrist','Left Elbow','Right Elbow']

            # Drawing the detected pose with specified landmarks and lines
            for idx, landmark in enumerate(specific_landmarks):
                # Draw a circle around each specified landmark
                landmark_x = int(landmark.x * frame.shape[1])
                landmark_y = int(landmark.y * frame.shape[0])
                cv2.circle(frame, (landmark_x, landmark_y), 5, (0, 255, 0), -1)

            # Draw lines between the landmarks
            connections = [[0, 1], [0, 2], [1, 2], [1, 3], [2, 4], [3, 5], [4, 6], [5, 7], [6, 8], [1, 12], [12, 10], [2, 11], [11, 9]]
            for connection in connections:
                cv2.line(frame, (int(specific_landmarks[connection[0]].x * frame.shape[1]), int(specific_landmarks[connection[0]].y * frame.shape[0])),
                         (int(specific_landmarks[connection[1]].x * frame.shape[1]), int(specific_landmarks[connection[1]].y * frame.shape[0])), (0, 0, 255), 2)

            cv2_imshow(frame)

            print("Landmark Labels:")
            for idx, label in enumerate(labels):
                print(f"{label}: x={specific_landmarks[idx].x}, y={specific_landmarks[idx].y}")

            print("\n")
    # Else, skip the frame without processing


# CSV file creation



Make sure to change the file path in the code below for the video.


The code below processes the video, extracts pose landmarks and stores them in a csv file.

In [ ]:
import cv2
import mediapipe as mp
import csv

# Initialise MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Initialise a frame counter
frame_counter = 0

# Specify the indices and labels for the landmarks of interest
landmark_indices = [0, 12, 11, 23, 24, 25, 26, 27, 28, 15, 16, 13, 14]
landmark_labels = ['Nose', 'Right Shoulder', 'Left Shoulder', 'Left Hip', 'Right Hip', 'Left Knee', 'Right Knee', 'Left Foot', 'Right Foot', 'Left Wrist', 'Right Wrist', 'Left Elbow', 'Right Elbow']

# Prepare the CSV file for writing
csv_filename = "//content/pose_landmarks.csv"  # Update this path as necessary

# Open the CSV file for writing
with open(csv_filename, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    # Write the header
    csvwriter.writerow(['Landmark Name', 'X', 'Y', 'Frame Number'])

    # Setup video capture
    video_path = "/content/8 (2).mov"  # Update this path to your video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError("Cannot open video")

    while True:
        # Read frame-by-frame
        ret, frame = cap.read()
        if not ret:
            print("Error reading frame or end of video. Exiting...")
            break

        # Increment the frame counter
        frame_counter += 1

        # Process only every 5th frame to reduce processing time
        if frame_counter % 5 == 0:
            results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            if results.pose_landmarks:
                for idx, label in zip(landmark_indices, landmark_labels):
                    landmark = results.pose_landmarks.landmark[idx]
                    # Write each landmark to the CSV
                    csvwriter.writerow([
                        label,
                        landmark.x,
                        landmark.y,
                        frame_counter
                    ])

            # If you wish to display the frame, adjust the display function accordingly
            # cv2.imshow('Frame', frame)
            # if cv2.waitKey(1) & 0xFF == ord('q'):
            #     break

    # Release the capture once done
    cap.release()
    cv2.destroyAllWindows()
    print("Video processing and CSV writing completed.")


Error reading frame or end of video. Exiting...
Video processing and CSV writing completed.


Transformed CSV file

In [ ]:
import pandas as pd

# Read the generated CSV file containing pose landmarks data.
df = pd.read_csv('/content/pose_landmarks.csv')

# Transform the dataframe for easier analysis.
# Pivot the table to make each landmark name a separate column for X and Y coordinates.
pivot_x = df.pivot(index='Frame Number', columns='Landmark Name', values='X').add_prefix('X_')
pivot_y = df.pivot(index='Frame Number', columns='Landmark Name', values='Y').add_prefix('Y_')
pivot_merged = pivot_x.merge(pivot_y, left_index=True, right_index=True)

# Save the transformed dataframe to a new CSV file
transformed_csv_filename = "transformed_pose_landmarks.csv"
pivot_merged.to_csv(transformed_csv_filename)


After the original CSV file is transformed the data "transformed_pose_landmarks.csv" is ready to be merged with anottation labels